<a href="https://colab.research.google.com/github/arnauldnzegha/face_fingerprinter_fusion_for_authentification/blob/master/fingerprint_recognition_using_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
##*****************LOAD AND UNZIP******************************
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile, os
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
zip_id = '11FkzDhCWZrXxlIn_CZ2SEUlv0riz8j_E'
if not os.path.exists('DATAF'):
    os.makedirs('DATAF')
myzip = drive.CreateFile({'id': zip_id})
myzip.GetContentFile('model.zip')
zip_ref = zipfile.ZipFile('model.zip', 'r')
zip_ref.extractall('DATAF/')
zip_ref.close()

In [0]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_lfw_people
from keras.utils import np_utils
from PIL import Image
import cv2
import numpy as np
import warnings
import matplotlib.pyplot as plt
from keras.models import Model
from keras.optimizers import SGD,Adadelta
from keras.layers import Flatten, Dense, Convolution2D, MaxPooling2D, Input, Dropout
from keras.layers.normalization import BatchNormalization
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file
from keras import backend as K
from random import shuffle
from keras.preprocessing.image import load_img, img_to_array, array_to_img

Using TensorFlow backend.


In [0]:
def load_trainSet():
    root="DATAF/VeriFinger_Sample_DB/"
    f_id=[12,13,17,22,27,45,47,57,76]
    bin_id=np.zeros([9,8])
    tab_id=[]
    x_data,y_label=[],[]
    for i in range (len(f_id)):
      for j in range(8):
        bin_id=np.zeros([9,8])
        bin_id[i][j]=1
        file_id="0"+str(f_id[i])+"_"+str(j+1)
        for n in range(1,9):
          file_name=root+file_id+"_"+str(n)+".tif"
          if os.path.exists(file_name):
            img=cv2.imread(file_name)
            x_data.extend([img])
            y_label.extend([bin_id.flatten()])
    return (np.asarray(x_data), np.asarray(y_label), 8*9) 

In [0]:
def mixData(xs,ys):
    xys=[]
    for i in range(0,len(xs)-1):
        xys.append((xs[i],ys[i]))
    shuffle(xys)
    x2,y2=[],[]
    for (x,y) in xys:
        x2.append(x)
        y2.append(y)
    return (np.asarray(x2), np.asarray(y2))
#X_train, Y_train=mixData(X_train, Y_train)

In [0]:
def save_weights_in_drive(model, filename):
  from google.colab import files
  model.save_weights('my_model_weights_fam.h5')
  uploaded = drive.CreateFile({'title': 'my_model_weights_fam.h5'})
  uploaded.SetContentFile('model_weights_AUG_300.h5')
  uploaded.Upload()
  print('Uploaded file with ID {}'.format(uploaded.get('id')))

In [0]:
def fr_model(input_shape,nb_person=1):
    img_input = Input(shape=(480, 504, 3))
    #Block 1 avec 64 filtre
    x = Convolution2D(64, 3, 3, activation='relu', border_mode='same', name='conv1_1')(img_input)
    x=BatchNormalization()(x)
    x = Convolution2D(64, 3, 3, activation='relu', border_mode='same', name='conv1_2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pool1')(x)
    x= Dropout(0.25)(x)

    # Block 2
    x = Convolution2D(128, 3, 3, activation='relu', border_mode='same', name='conv2_1')(x)
    x = Convolution2D(128, 3, 3, activation='relu', border_mode='same', name='conv2_2')(x)
    x=BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pool2')(x)
    x= Dropout(0.25)(x)

    # Block 3
    x = Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='conv3_1')(x)
    x = Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='conv3_2')(x)
    x=BatchNormalization()(x)
    x = Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='conv3_3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pool3')(x)
    x= Dropout(0.25)(x)

    # Block 4
    x = Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='conv4_1')(x)
    x = Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='conv4_2')(x)
    x = Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='conv4_3')(x)
    x=BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pool4')(x)
    x= Dropout(0.25)(x)

    # Block 5
    x = Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='conv5_1')(x)
    x = Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='conv5_2')(x)
    x = Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='conv5_3')(x)
    x=BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pool5')(x)
    x= Dropout(0.25)(x)

    #Classifieur du modele
    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='relu', name='fc6')(x)
    x= Dropout(0.25)(x)
    x = Dense(4096, activation='relu', name='fc7')(x)
    x = Dense(nb_person, activation='softmax', name='fc8')(x)
    model = Model(img_input, x)
    return model

In [0]:
if __name__ == '__main__':
    import copy
    (x_train, y_train, nb_class)=load_trainSet()
    print(x_train[100].shape)
    (x_train, y_train)=mixData(x_train, y_train)
    model = fr_model(nb_person=nb_class, input_shape=x_train[0].shape)
    #sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    delta=Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=delta,  metrics=['accuracy'])
    print("Entrainement du modele")
    hist=model.fit(x_train, y_train, validation_split=0.33, batch_size=10, epochs=100, verbose=1)

In [0]:
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('courbe de la perte DataSet avec augmentation / 1000 itération')
plt.ylabel('Perte')
plt.xlabel('itération')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
(x, _, _)=load_trainSet()
test=model.predict(x[14:44])
list_id=np.argmax(test, axis=1)
print(list_id)